<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Insurance Analysis Using Temporal</b>
</header>


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:18px;font-family:Arial'>
Applications require to design and build databases dependant on time or data changes over time.  Building such applications is possible but becomes easier with Temporal
   
</p>  

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>WHY TEMPORAL? </b> </p>

<p style = 'font-size:18px;font-family:Arial'>    
Things You No Longer Have To Do in the Time-Aware Teradata Database
Data Maintenance : </p>
    
<li style = 'font-size:16px;font-family:Arial'> Update effective dates in data.</li></<li style = 'font-size:16px;font-family:Arial'> <li style = 'font-size:16px;font-family:Arial'> Specify “transaction time” timestamp when data are entered or updated. </li>
<li style = 'font-size:16px;font-family:Arial'> Insert new data rows for updates. </li>     
<li style = 'font-size:16px;font-family:Arial'> Manage temporal history storage space. </li> 
  

 
<p style = 'font-size:18px;font-family:Arial'> Query and Application Development: </p>


<li style = 'font-size:16px;font-family:Arial'> Write complex period based condition clauses. </li>
<li style = 'font-size:16px;font-family:Arial'> Add effective date qualifications to “current queries.” </li>
<li style = 'font-size:16px;font-family:Arial'> Train users and application developers in effective date columns and processing for “AS OF” query development.</li>
<li style = 'font-size:16px;font-family:Arial'> Save old reports or database snapshots to facilitate looking back for analysis, compliance reporting, or audit trails.</li>

<p style = 'font-size:16px;font-family:Arial'>Here we are considering the example of Insurance company storing data for various policies whose validity, conditions , premium amount and many other factors will change over Time period.</p>
    
    
 

   
    
    
    
<p style = 'font-size:18px;font-family:Arial'>Some scenarios that can be handled using temporal features are as below 
    
<li style = 'font-size:16px;font-family:Arial'>    Consider the scenario where there is a hail storm striking on a particular date. The As of date feature can be used to check the validity of the policy and then the incident date can be used to check which claims fall in the validity period of the policies.</li>
    
<li style = 'font-size:16px;font-family:Arial'>    Another scenario is that the customer has sold the vehicle and the closure of the policy happened a week late as the customer informed late about the sale , so based on the transaction time the validity of the policies will change. </li>



<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>1. Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [ ]:
SET query_band='DEMO=Insurance_Policy_Temporal_SQL.ipynb;' UPDATE FOR SESSION;

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>2. Getting Data for This Demo:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>

In [ ]:
call get_data('DEMO_Insurance_local');    -- Takes about 90 seconds

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
call space_report();  -- optional, takes about 10 seconds

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>3. Comparison of Temporal and Non-Temporal:</b></p>
<p style = 'font-size:16px;font-family:Arial'>To understand how Temporal can be helpful we will consider a simple example where we need to check policies which were valid on 31st December 2020 and had a premium amount less than 1000. for a normal table with start and end date columns the query will be as below:</p>

In [ ]:
SELECT
    Policy_ID,
    Customer_ID,
    Policy_premium_Amt,
    policy_state,
    Policy_start_date,
    policy_end_date
FROM
    DEMO_Insurance.Policy_Details_Org
WHERE
    policy_start_date <= '2020-12-31'
    AND policy_end_date >= '2020-12-31'
    AND Policy_premium_Amt < 1000;

<p style = 'font-size:16px;font-family:Arial'>The same query with Temporal table would be as below:</p>

In [ ]:
VALIDTIME AS OF DATE '2020-12-31'
SELECT
    Policy_ID,
    Customer_ID,
    Policy_premium_Amt,
    policy_state,
    Policy_Validity
FROM
    DEMO_Insurance.Policy_Details
WHERE
    Policy_premium_Amt < 1000;

<p style = 'font-size:18px;font-family:Arial'><b> Now let's check how temporal can help simplify queries in a complex scenario:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Consider scenario where we have to check all valid policies whose validity period is similar to the Policy of the Mercedes -ML350 in the state IL. In a table without the temporal columns the query would have to collect data for various scenarios where the policy_start_date and policy_end_date for other policies lie in the range of the policy_start_date and policy_end_date of the policy of Mercedes-ML350 in state of IL. So the query will have to do a UNION ALL with different date checks for checking the start and end date of the policies looks like the below: </p>

In [ ]:
SELECT
    OthPol.Policy_id,
    OthPol.Customer_id,
    OthPol.auto_make,
    OthPol.Policy_start_date,
    OthPol.Policy_end_date
FROM
    DEMO_Insurance.policy_details_org MercPol
JOIN
    DEMO_Insurance.policy_details_org OthPol
ON
    MercPol.auto_make = 'Mercedes'
    AND MercPol.policy_state = 'IL'
    AND MercPol.auto_model = 'ML350'
    AND MercPol.policy_state = OthPol.Policy_state
    AND MercPol.Policy_start_date <= OthPol.Policy_start_date
    AND MercPol.Policy_end_date > OthPol.Policy_start_date
    AND MercPol.Policy_end_date <= OthPol.Policy_end_date
UNION ALL
SELECT
    OthPol.Policy_id,
    OthPol.Customer_id,
    OthPol.auto_make,
    OthPol.Policy_start_date,
    OthPol.Policy_end_date
FROM
    DEMO_Insurance.policy_details_org MercPol
JOIN
    DEMO_Insurance.policy_details_org OthPol
ON
    MercPol.auto_make = 'Mercedes'
    AND MercPol.policy_state = 'IL'
    AND MercPol.auto_model = 'ML350'
    AND MercPol.policy_state = OthPol.Policy_state
    AND MercPol.Policy_start_date <= OthPol.Policy_start_date
    AND MercPol.Policy_end_date >= OthPol.Policy_end_date
UNION ALL
SELECT
    OthPol.Policy_id,
    OthPol.Customer_id,
    OthPol.auto_make,
    OthPol.Policy_start_date,
    OthPol.Policy_end_date
FROM
    DEMO_Insurance.policy_details_org MercPol
JOIN
    DEMO_Insurance.policy_details_org OthPol
ON
    MercPol.auto_make = 'Mercedes'
    AND MercPol.policy_state = 'IL'
    AND MercPol.auto_model = 'ML350'
    AND MercPol.policy_state = OthPol.Policy_state
    AND MercPol.Policy_start_date >= OthPol.Policy_start_date
    AND MercPol.Policy_end_date <= OthPol.Policy_end_date
UNION ALL
SELECT
    OthPol.Policy_id,
    OthPol.Customer_id,
    OthPol.auto_make,
    OthPol.Policy_start_date,
    OthPol.Policy_end_date
FROM
    DEMO_Insurance.policy_details_org MercPol
JOIN
    DEMO_Insurance.policy_details_org OthPol
ON
    MercPol.auto_make = 'Mercedes'
    AND MercPol.policy_state = 'IL'
    AND MercPol.auto_model = 'ML350'
    AND MercPol.policy_state = OthPol.Policy_state
    AND MercPol.Policy_start_date >= OthPol.Policy_start_date
    AND MercPol.Policy_start_date < OthPol.Policy_end_date
    AND MercPol.Policy_end_date >= OthPol.Policy_end_date;

<p style = 'font-size:16px;font-family:Arial'>Use the Temporal ValidTime function to display similar results as the query written above on table without Period datatype. </p>

In [ ]:
VALIDTIME SELECT
    OthPol.Policy_id,
    OthPol.Customer_id,
    OthPol.auto_make
FROM
    DEMO_Insurance.policy_details MercPol
JOIN
    DEMO_Insurance.policy_details OthPol
ON
    MercPol.auto_make = 'Mercedes'
    AND MercPol.policy_state = 'IL'
    AND MercPol.auto_model = 'ML350'
    AND MercPol.policy_state = OthPol.Policy_state;

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>4. Comparison for Queries with and without Temporal </b></p>

<table>
<tr>
<td style="color:black;font-size:18px;font-weight:bold">WITH TEMPORAL</td>
<td style="color:black;font-size:18px;font-weight:bold">WITHOUT TEMPORAL</td> 
</tr>

<tr>
<td>Validtime select b.Policy_id, b.Customer_id, b.auto_make from policy_details a , policy_details b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state; </td>
<td>select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date b.policy_start_date AND a.policy_end_date    UNION ALL 
select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date <= b.policy_start_date AND a.policy_end_date >= b.policy_end_date

   UNION ALL 
select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date >= b.policy_start_date AND a.policy_end_date <= b.policy_end_date

   UNION ALL
select b.Policy_id, b.Customer_id, b.auto_make , b.Policy_start_date,b.Policy_end_date from policy_details_org_extrapolate a , policy_details_org_extrapolate b where a.auto_make = 'Mercedes' and a.policy_state = 'IL' and a.auto_model = 'ML350' and a.policy_state = b.Policy_state AND a.policy_start_date >= b.policy_start_date AND a.policy_start_date < b.policy_end_date AND a.policy_end_date >= b.policy_end_date;</td>
</tr>    
</table>    
 

 <p style = 'font-size:18px;font-family:Arial;'> <b>Performance Comparison for above queries </b> </p>

<img src="images/PerformanceMetrics.png" style="width:1000px;height:200px;">

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>5. Temporal concepts:- Aggregations </b></p>

<p style = 'font-size:16px;font-family:Arial'>Comparing the Validity and VALIDTIME columns demonstrates that the   
    valid-time period of the result set is the intersection of the valid-time period in the original Policy table Validity 
    column with the Period specified in the sequenced select statement.</p>

In [ ]:
SEQUENCED VALIDTIME PERIOD '(2009-01-01, 2009-12-31)'
SELECT
    Policy_ID,
    Customer_ID,
    Policy_Validity
FROM
    (
        SEQUENCED VALIDTIME SELECT
            Policy_details.*
        FROM
            DEMO_Insurance.Policy_Details
        WHERE
            policy_state = 'IN'
    ) AS my_derived_table;

<p style = 'font-size:16px;font-family:Arial'>Aggregations can be used for analysis of the data based on the validity period. 
Comparisons of data between different validity periods can be achieved with Temporal. Like the minimum and maximum premium amounts for a particular state based on the model of the cars.</p>



In [ ]:
SEQUENCED VALIDTIME PERIOD (DATE '2011-01-01', DATE '2020-12-31')
SELECT
    auto_make,
    MIN(Policy_premium_Amt) AS MinPremium,
    MAX(Policy_premium_Amt) AS MaxPremium
FROM
    DEMO_Insurance.Policy_Details
WHERE
    policy_state = 'OH'
GROUP BY 1
ORDER BY VALIDTIME;

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>6. Temporal concepts:- Validtime usage </b></p>

<p style = 'font-size:18px;font-family:Arial'> <b> Time line for various Policies:-    Analysis can be done for </b> </p>
    
   
<li style = 'font-size:16px;font-family:Arial'>Policies valid during a particular period.</li>
    
<li style = 'font-size:16px;font-family:Arial'>Validity period of various policies.</li>
    
<li style = 'font-size:16px;font-family:Arial'>Premium amount collected during a particular period </li>

<p style = 'font-size:18px;font-family:Arial'>Using the As Of concept to verify the active policies on a particular date</p>

<p style = 'font-size:16px;font-family:Arial'>On a given date, there was a hailstorm and we want to determine the count of policies in effect on that date
The scenario mentioned here can be solved by the below query using the As of Date feature of Temporal tables   
 </p>


<img src="images/TemporalTimePeriod.png" style="width:600px;height:400px;">



In [ ]:
VALIDTIME AS OF DATE '2022-02-15'
SELECT
    Policy_ID,
    Customer_ID,
    Policy_premium_Amt,
    Policy_Validity
FROM
    DEMO_Insurance.Policy_Details
WHERE
    policy_state = 'IL';

In [ ]:
SELECT
    COUNT(*) AS "no of Policies"
FROM
    (
        VALIDTIME AS OF DATE '2022-02-15'
        SELECT
            Policy_ID,
            Customer_ID,
            Policy_premium_Amt,
            Policy_Validity
        FROM
            DEMO_Insurance.Policy_Details
        WHERE
            policy_state = 'IL'
    ) AS DerivTbl;

<p style = 'font-size:16px;font-family:Arial'>Consider scenario where we need to check that for a particular period (here for year 2021) how many policies are valid for each auto make type and then compare the no of policies which are valid in the current period. This helps us in the comparison in the increase or decrease of policies for the current year as compared to the past year of validity
 </p>


In [ ]:
SELECT
    auto_make,
    COUNT(*) AS "no of Policies"
FROM
    (
        VALIDTIME PERIOD (DATE '2021-01-01', DATE '2021-12-31')
        SELECT
            Policy_ID,
            Customer_ID,
            Policy_premium_Amt,
            auto_make
        FROM
            DEMO_Insurance.Policy_Details
    ) AS DerivTbl
GROUP BY
    auto_make;

In [ ]:
%chart x = auto_make, y = "no of Policies", title = 'Valid Number of Policies last year'

<p style = 'font-size:16px;font-family:Arial'>Now we check how many policies are still valid for each type using the current functionality
 </p>


In [ ]:
SELECT
    auto_make,
    COUNT(*) AS "no of Policies"
FROM
    (
        CURRENT VALIDTIME
        SELECT
            Policy_ID,
            Customer_ID,
            Policy_premium_Amt,
            auto_make
        FROM
            DEMO_Insurance.Policy_Details
    ) AS DerivTbl
GROUP BY
    auto_make;

In [ ]:
%chart x = auto_make, y = "no of Policies", title = 'Current Valid Number of Policies'

<p style = 'font-size:16px;font-family:Arial'>In this scenario we are considering that there are a number of claims raised against policies on various incident dates. Below queries help to identify which claims made are in the validity period and which are not and which policies are valid at the time of the incidents. There are total 69 claims in the Insurance Claims tables and based on the incident date we can verify which claims are valid and which are not valid.
 </p>


In [ ]:
SELECT * FROM DEMO_Insurance.Insurance_Claims_Details;

<p style = 'font-size:16px;font-family:Arial'>Based on the Claims made, we need to validate which claims are valid on the incident date mentioned. Below query helps in the analysis. Only the rows that qualify for the claims are displayed.
 </p>


In [ ]:
SELECT
    'Valid Claims' AS ClaimType,
    Policy.customer_id,
    Policy.Policy_id,
    Insurance_Claims_Details.Claim_Id,
    Claim_amount
FROM
    (
        CURRENT VALIDTIME
        SELECT
            Policy_Details.*
        FROM
            DEMO_Insurance.Policy_Details
    ) Policy
INNER JOIN
    DEMO_Insurance.Insurance_Claims_Details
ON
    Policy.Policy_id = Insurance_Claims_Details.Policy_id
WHERE
    Policy.Policy_Validity CONTAINS Insurance_Claims_Details.Incident_date;

<p style = 'font-size:16px;font-family:Arial'>Below query displays the Invalid rows from Claims table
 </p>


In [ ]:
SELECT
    customer_id,
    Policy.Policy_id,
    Claim_Id,
    Claim_amount
FROM
    DEMO_Insurance.Insurance_Claims_Details
INNER JOIN
    DEMO_Insurance.Policy_Details Policy
ON
    Policy.Policy_id = Insurance_Claims_Details.Policy_id
MINUS
SELECT
    Policy.customer_id,
    Policy.Policy_id,
    Insurance_Claims_Details.Claim_Id,
    Claim_amount
FROM
    (
        CURRENT VALIDTIME
        SELECT
            Policy_Details.*
        FROM
            DEMO_Insurance.Policy_Details
    ) Policy
INNER JOIN
    DEMO_Insurance.Insurance_Claims_Details
ON
    Policy.Policy_id = Insurance_Claims_Details.Policy_id
WHERE
    Policy.Policy_Validity CONTAINS Insurance_Claims_Details.Incident_date;

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>7. Explaining the Temporal Concepts of Inserts /Deletes </b></p> 

<p style = 'font-size:16px;font-family:Arial'>Customer 1 has policy 1 with Validity Period . In between the validity period the customer decides to change some factors which causes the premium to increase , so there will be a new addition which will insert a new row having validity period from changed date to the end of period. </p>
    
    
    
<p style = 'font-size:16px;font-family:Arial'>Customer 2 has Policy 1 with validity period. In between the company decides to increase the premium which also increases the validity period.
 </p>


<img src="images/Temporal_Inserts_Updates.png" style="width:600px;height:400px;">



<p style = 'font-size:18px;font-family:Arial'>Inserting data into temporal table is simple for current validtime. </p>

In [ ]:
CURRENT VALIDTIME INSERT DEMO_Insurance_DB.DEMO_Insurance_Policy_Det
VALUES
    (203914, 2316038, 'OH', 1200, 'Audi', 'A3');

<p style = 'font-size:18px;font-family:Arial'>Inserting history rows will need Temporal Qualifier. </p>

In [ ]:
SEQUENCED VALIDTIME INSERT DEMO_Insurance_DB.DEMO_Insurance_Policy_Det
VALUES
    (203914, 2316038, 'OH', 1050, 'Audi', 'A3', PERIOD(DATE '2002-06-10', DATE '2012-06-09'));

SEQUENCED VALIDTIME INSERT DEMO_Insurance_DB.DEMO_Insurance_Policy_Det
VALUES
    (203914, 2316038, 'OH', 1150, 'Audi', 'A3', PERIOD(DATE '2012-06-10', DATE '2022-06-27'));

In [ ]:
SEQUENCED VALIDTIME
SELECT * 
FROM
    DEMO_Insurance.Policy_Details 
WHERE
    customer_id = 2316038 
ORDER BY
    validtime;

<p style = 'font-size:16px;font-family:Arial'>Without Temporal Qualifier the select will display only current rows. </p>

In [ ]:
SET SESSION CURRENT VALIDTIME;

In [ ]:
SELECT *
FROM 
    DEMO_Insurance.Policy_Details
WHERE
    customer_id = 2316038
ORDER BY
    Policy_validity;

<p style = 'font-size:16px;font-family:Arial'>When period is specified , Delete will only delete rows for that specific period (highlighted row in the above dataset) and the validity period for rows will change to the end of period specified in the delete. </p> 

<p style = 'font-size:16px;font-family:Arial'>Display rows for Specific Valid Period.</p>

In [ ]:
SEQUENCED VALIDTIME PERIOD '(1990-01-01, 1990-03-31)'
SELECT * 
FROM
    DEMO_Insurance.Policy_details
WHERE
    policy_state = 'IN'
ORDER BY
    validtime;

<p style = 'font-size:16px;font-family:Arial'>Delete rows for Specific Valid Period will change the validity period to the end date specified in delete. </p>

In [ ]:
SEQUENCED VALIDTIME PERIOD '(1990-01-01, 1990-01-30)'
DELETE FROM
    DEMO_Insurance_DB.DEMO_Insurance_Policy_Det
WHERE
    policy_state = 'IN';

In [ ]:
SEQUENCED VALIDTIME PERIOD '(1990-01-01, 1990-03-31)'
SELECT * 
FROM
    DEMO_Insurance_DB.DEMO_Insurance_Policy_Det
WHERE
    policy_state = 'IN'
ORDER BY
    validtime;

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>8. Bi-Temporal Table with ValidTime and TransactionTime Columns and its usage.</b> </p>

<p style = 'font-size:16px;font-family:Arial'>Bi-Temporal tables or Transaction time tables have both Valid Date(Date) Columns and Transaction Time (Timestamp) Columns. Transaction time is the time period during which a fact, represented by all the information in a row, is or was known to be in effect in the database. It models the database reality, recording when rows have been added, modified, and changed in the database. Transaction-time periods are stored in a transaction-time column:</p>

<li style = 'font-size:16px;font-family:Arial'>The beginning of the transaction-time period is the time when the database became aware of a row, when the row was first recorded in the database. This is when the row was added to a table.</li>
<li style = 'font-size:16px;font-family:Arial'>The end of a transaction time period reflects when the fact was superseded by an update to the row, or when the row was deleted from the database. Rows containing information that is currently in effect have transaction-time periods with indefinite ending bounds, represented as UNTIL_CLOSED.</li>



<p style = 'font-size:16px;font-family:Arial'>Thus, temporal features can help in simplifying time dependant applications. Temporal and Bi-Temporal tables can be analyzed and used in various scenarios.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>9. Cleanup </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_Insurance');    -- Takes about 10 seconds, optional if you want to use the data later

</h4>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Information about Temporal datatype can be found at: <a href = 'https://docs.teradata.com/search/all?query=Temporal+Table+Support&content-lang=en-US'>https://docs.teradata.com/search/all?query=Temporal+Table+Support&content-lang=en-US</a></li>
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>